In [12]:
import requests

url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
data = {
    "chat_id": "1",
    "messages": [{"type": "text", "content": "return base random key: 123e4567-e89b-12d3-a456-426614174000" }]
}

# Disable system proxy
resp = requests.post(url, json=data, proxies={"http": None, "https": None})

In [13]:
resp.json()

{'message': None,
 'base_random_keys': ['123e4567-e89b-12d3-a456-426614174000'],
 'member_random_keys': None}